In [1]:
from HanGrab import *

In [3]:
UpdateData(src='han', start=109)


[False, False, False, False, False, False, False, False]
updating  8 hangame files
66982
66997
67041


KeyboardInterrupt: 

In [19]:
UpdateData(src='oro', start=298)

2
updating  15  oro files
3
updating  15  oro files
4
updating  15  oro files
5
updating  15  oro files
6
updating  15  oro files
7
updating  15  oro files
8
updating  15  oro files
9
updating  15  oro files
10
updating  15  oro files
11
updating  15  oro files
12
updating  15  oro files
13
updating  15  oro files
14
updating  15  oro files
15
updating  15  oro files
16
updating  15  oro files
17
updating  15  oro files
18
updating  15  oro files
19
updating  15  oro files
20
updating  15  oro files
21
updating  15  oro files
22
updating  15  oro files
23
updating  15  oro files
24
updating  15  oro files
25
updating  15  oro files
26
updating  15  oro files
27
updating  15  oro files
28
updating  15  oro files
29
updating  15  oro files
30
updating  15  oro files
31
updating  15  oro files
32
updating  15  oro files
33
updating  15  oro files
34
updating  15  oro files
35
updating  15  oro files
36
updating  15  oro files
37
updating  15  oro files
38
updating  15  oro files
39
updati

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


212
updating  15  oro files
213
updating  15  oro files
214
updating  15  oro files
215
updating  15  oro files
216
updating  15  oro files
217
updating  15  oro files
218
updating  15  oro files
219
updating  15  oro files
220
updating  15  oro files
221
updating  15  oro files
222
updating  15  oro files
223
updating  15  oro files
224
updating  15  oro files
225
updating  15  oro files
226
updating  15  oro files
227
updating  15  oro files
228
updating  15  oro files
229
updating  15  oro files
230
updating  15  oro files
231
updating  15  oro files
232
updating  15  oro files
233
updating  15  oro files
234
updating  15  oro files
235
updating  15  oro files
236
updating  15  oro files
237
updating  15  oro files
238
updating  15  oro files
239
updating  15  oro files
240
updating  15  oro files
241
updating  15  oro files
242
updating  15  oro files
243
updating  15  oro files
244
updating  15  oro files
245
updating  15  oro files
246
updating  15  oro files
247
updating  15  or

ConnectionError: HTTPConnectionPool(host='cyberoro.com', port=80): Max retries exceeded with url: /news/news_list.oro?cmt_n=0&div_no=A1&pageNo=299&blockNo=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f5762c510d0>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [4]:
UpdateData(src='tygem', start=32, sleep=4)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
updating  13 tygem files


KeyboardInterrupt: 